<a href="https://colab.research.google.com/github/Maryam-Mostafa/Natural-scene-classification-using-different-CNN-architecture/blob/master/inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tflearn
import os
import numpy as np
import pandas as pd
import cv2
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
import tflearn
from keras.utils import np_utils
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from sklearn.model_selection import train_test_split
from keras.layers import *
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
def Read_Data(path_,label,isTrain):
    sub_data = []
    for img in tqdm(os.listdir(path_)):
        path = os.path.join(path_, img)
        img_data = cv2.imread(path, 0)
        img_data = cv2.resize(img_data, (150, 150))
        if isTrain:
          sub_data.append([np.array(img_data), label])
        else:
          sub_data.append([np.array(img_data), img])
    return sub_data

def Creat_all_train_data():
    training_data,sub1,sub2,sub3,sub4,sub5,sub6 = [],[],[],[],[],[],[]
          
    sub1 = Read_Data("/content/drive/MyDrive/NNP/Scenetrainingset/buildings/",0,True)
    sub2 = Read_Data("/content/drive/MyDrive/NNP/Scenetrainingset/forest",1,True)
    sub3 = Read_Data("/content/drive/MyDrive/NNP/Scenetrainingset/glacier",2,True)
    sub4 = Read_Data("/content/drive/MyDrive/NNP/Scenetrainingset/mountain",3,True)
    sub5 = Read_Data("/content/drive/MyDrive/NNP/Scenetrainingset/sea",4,True)
    sub6 = Read_Data("/content/drive/MyDrive/NNP/Scenetrainingset/street",5,True)
    training_data = sub1 + sub2 + sub3 + sub4 + sub5 + sub6
    shuffle(training_data)
    np.save('/content/drive/MyDrive/NNP/train_dataset.npy', training_data)

    return training_data

def inception_module(layer_in, f1, f2, f3):
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
    conv3 = Conv2D(f2, (3,3), padding='same', activation='relu')(layer_in)
    conv5 = Conv2D(f3, (5,5), padding='same', activation='relu')(layer_in)
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    return layer_out

if (os.path.exists('/content/drive/MyDrive/NNP/train_dataset.npy')): 
    train =np.load('/content/drive/MyDrive/NNP/train_dataset.npy',allow_pickle=True)

else: 
    train = Creat_all_train_data()

if (os.path.exists('/content/drive/MyDrive/NNP/test_dataset.npy')): 
    test =np.load('/content/drive/MyDrive/NNP/test_dataset.npy',allow_pickle=True)

else: 
      test = Read_Data("/content/drive/MyDrive/NNP/Scenestestingtest",None,False)
      np.save('/content/drive/MyDrive/NNP/test_dataset.npy', test)

X_train = np.array([i[0] for i in train]).reshape(-1, 150, 150, 1)
Y_train = np.array([i[1] for i in train])
Y_train = np_utils.to_categorical(Y_train, num_classes=6)


X_test = np.array([i[0] for i in test]).reshape(-1, 150, 150, 1)
img_name = np.array([i[1] for i in test])

#csvFile = pd.read_csv("/content/drive/MyDrive/NNP/submit.csv")

ops.reset_default_graph()

conv_input = input_data(shape=[None, 150, 150, 1], name='input')

inception1 = inception_module(conv_input , 32, 64, 32)
inception2 = inception_module(inception2 , 32, 64, 32)

fully_layer = fully_connected(inception2, 500, activation='relu')
fully_layer = dropout(fully_layer, 0.5)

cnn_layers = fully_connected(inception2, 6, activation='softmax')

cnn_layers = regression(cnn_layers, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy', name='targets')
model = tflearn.DNN(cnn_layers, tensorboard_dir='log', tensorboard_verbose=3)


model.fit( X_train, Y_train, n_epoch=1,snapshot_step=100, show_metric=True, run_id='cnn')
prediction = model.predict(X_test)
prediction = np.argmax(prediction, axis=1)
# print(prediction)

rows = {'Image': [],'Label': []}
csvFile = pd.DataFrame(rows, columns = ['Image', 'Label'])

csvFile["Image"] = img_name
csvFile["Label"] = np.asarray(prediction)
csvFile.to_csv('submit3.csv')
files.download('submit3.csv')

Instructions for updating:
non-resource variables are not supported in the long term
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
---------------------------------
Run id: cnn
Log directory: log/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 14034
Validation samples: 0
--
